Get your openai api key from settings: https://platform.openai.com/account/api-keys

In [187]:
# Library Installs
%pip install openai --upgrade
%pip install pydub simpleaudio
%pip install nltk
%pip install pyaudio numpy
%pip install sounddevice scipy
%pip install -U openai-whisper

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: pyaudio in c:\users\mattj\appdata\local\packages\pythonsoftwarefoundation.python.3.11_qbz5n2kfra8p0\localcache\local-packages\python311\site-packages (0.2.14)



Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


In [188]:
# Library Imports
import getpass
from openai import OpenAI
import os



In [198]:
# Configuration Variables
MP3_FOLDER = "audio_chunks"  # Directory where audio chunks are stored
errorsLeft = True # Set as true to enter main program While loop

# MP3 Recording Parameters
SAMPLE_RATE = 44100  # Sample rate in Hz
DURATION_THRESHOLD = 1.5  # Duration in seconds to consider silence
SILENCE_THRESHOLD = 0.01  # Amplitude threshold for silence
MAX_CHUNK_SIZE = 25 * 1024 * 1024  # Maximum chunk size in bytes (25 MB)



In [199]:
# Play Audio Function
def play_audio(mp3_filename):
    """
    Plays the specified MP3 file using pydub and simpleaudio.
    
    :param mp3_filename: Name of the MP3 file to play (should be located in the current working directory)
    """
    try:
        # Load audio file
        audio = AudioSegment.from_mp3(Path.cwd() / mp3_filename)
        
        # Play the audio file
        play_obj = sa.play_buffer(
            audio.raw_data,
            num_channels=audio.channels,
            bytes_per_sample=audio.sample_width,
            sample_rate=audio.frame_rate
        )
        play_obj.wait_done()  # Wait until playback is finished
        print(f"Finished playing {mp3_filename}")
    except Exception as e:
        print(f"Error playing audio: {e}")

In [ ]:
# Records user audio and detects pauses (silence) to segment the recording into separate chunks.
# When silence is detected for a specified duration, the audio is saved as an MP3 file. This allows for efficient 
# segmentation of audio data based on user pauses. The program also ensures that audio chunks do not exceed a set size.

import sounddevice as sd
import numpy as np
import os
import time
from pydub import AudioSegment
from scipy.io.wavfile import read


# Create directory for audio chunks if it doesn't exist
if not os.path.exists(MP3_FOLDER):
    os.makedirs(MP3_FOLDER)

def play_ding():
    """
    Play a hardcoded 'ding.wav' file 
    """
    file_path = "ding.wav"  # Hardcoded file name
    
    # Read the WAV file
    sample_rate, data = read(file_path)
    
    # Play the audio file
    sd.play(data, sample_rate)
    sd.wait()  # Wait until playback is finished

def is_silent(data):
    """Check if the audio data is silent by evaluating if its mean amplitude is below the silence threshold."""
    return np.abs(data).mean() < SILENCE_THRESHOLD
"""
def record_audio():
    """Continuously record audio, saving each chunk when silence is detected for the threshold duration."""
    print("Recording... Speak now.")
    play_ding()

    # Initialize the current chunk and silence detection timing
    current_chunk = []
    silence_start_time = None

    # Start the audio input stream
    with sd.InputStream(samplerate=SAMPLE_RATE, channels=1, dtype='float32') as stream:
        while True:
            # Read one second of audio data from the stream
            data = stream.read(SAMPLE_RATE)[0]
            current_chunk.extend(data.flatten())  # Add the new data to the current chunk

            # Check if the audio data is silent
            if is_silent(data):
                # If silence starts, begin a timer
                if silence_start_time is None:
                    silence_start_time = time.time()
            else:
                # Reset the timer if sound is detected
                silence_start_time = None

            # Stop recording if silence has lasted beyond the duration threshold
            if silence_start_time and (time.time() - silence_start_time) > DURATION_THRESHOLD:
                print("Silence detected. Stopping recording.")
                break  # Exit the loop to stop recording

    # Save the final chunk after breaking out of the loop
    if current_chunk:
        save_audio_chunk(current_chunk)
        print("Final chunk saved.")

def save_temp_wav(file_path, chunk):
    """Save the recorded audio chunk to a WAV file temporarily for further processing or conversion."""
    if len(chunk) == 0:
        return  # Don't save empty chunks
    
    # Convert to WAV and save
    audio_data = np.array(chunk, dtype=np.float32)
    write(file_path, SAMPLE_RATE, audio_data)

def convert_to_mp3(wav_file, mp3_file):
    """Convert a temporary WAV file to MP3 format using the Pydub library."""
    try:
        audio = AudioSegment.from_wav(wav_file)
        audio = audio.set_channels(1)  # Ensure mono channel
        audio = audio.set_frame_rate(16000)  # Set sample rate to 16 kHz (Whisper's recommended rate)
        audio.export(mp3_file, format="mp3", bitrate="128k")  # Set bitrate to 128 kbps
    except Exception as e:
        print(f"Error converting to MP3: {e}")


def save_audio_chunk(chunk):
    """Save the recorded audio chunk to an MP3 file, creating a unique file name."""
    if len(chunk) == 0:
        return  # Don't save empty chunks

    # Create a unique file name for the MP3 file
    chunk_index = len(os.listdir(MP3_FOLDER)) + 1
    mp3_file_path = os.path.join(MP3_FOLDER, f"chunk{chunk_index}.mp3")

    # Convert the chunk to WAV temporarily for saving
    temp_wav_path = os.path.join(MP3_FOLDER, "temp.wav")
    save_temp_wav(temp_wav_path, chunk)

    # Convert to MP3
    convert_to_mp3(temp_wav_path, mp3_file_path)

    print(f"Saved chunk to {mp3_file_path}")

"""

In [ ]:
def record_audio():
    """Record audio until silence is detected for the threshold duration, then save as a single MP3."""
    print("Recording... Speak now.")
    play_ding()

    # Initialize the audio buffer and silence detection timing
    audio_buffer = []
    silence_start_time = None

    # Start the audio input stream
    with sd.InputStream(samplerate=SAMPLE_RATE, channels=1, dtype='float32') as stream:
        while True:
            # Read one second of audio data from the stream
            data = stream.read(SAMPLE_RATE)[0]
            audio_buffer.extend(data.flatten())  # Add the new data to the audio buffer

            # Check if the audio data is silent
            if is_silent(data):
                # Start silence timer if silence detected
                if silence_start_time is None:
                    silence_start_time = time.time()
            else:
                # Reset silence timer if sound resumes
                silence_start_time = None

            # Stop recording if silence lasts beyond the duration threshold
            if silence_start_time and (time.time() - silence_start_time) > DURATION_THRESHOLD:
                print("Silence detected. Stopping recording.")
                break  # Exit the loop to stop recording

    # Save the final audio as an MP3 after recording stops
    if audio_buffer:
        save_audio_as_mp3(audio_buffer)
        print("Audio saved.")

def save_audio_as_mp3(audio_buffer):
    """Convert and save the audio buffer to an MP3 file."""
    # Create a unique file name for the MP3 file
    mp3_file_path = os.path.join(MP3_FOLDER, "recording.mp3")

    # Convert the buffer to a WAV file temporarily
    temp_wav_path = os.path.join(MP3_FOLDER, "temp.wav")
    audio_data = np.array(audio_buffer, dtype=np.float32)
    write(temp_wav_path, SAMPLE_RATE, audio_data)

    # Convert to MP3
    convert_to_mp3(temp_wav_path, mp3_file_path)

    print(f"Audio saved as {mp3_file_path}")

def convert_to_mp3(wav_file, mp3_file):
    """Convert a temporary WAV file to MP3 format using the Pydub library."""
    try:
        audio = AudioSegment.from_wav(wav_file)
        audio = audio.set_channels(1)  # Ensure mono channel
        audio = audio.set_frame_rate(16000)  # Set sample rate to 16 kHz (Whisper's recommended rate)
        audio.export(mp3_file, format="mp3", bitrate="128k")  # Set bitrate to 128 kbps
    except Exception as e:
        print(f"Error converting to MP3: {e}")

# Example usage
record_audio()  # Starts recording and will save as "recording.mp3" once silence is detected


In [201]:
# Trim Audio (dead air)
import os
from pydub import AudioSegment
from pydub.silence import detect_nonsilent

# Default configuration

def get_newest_file(directory):
    """Return the newest file in the specified directory."""
    files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
    if not files:
        return None
    files.sort(key=lambda f: os.path.getmtime(os.path.join(directory, f)), reverse=True)
    return os.path.join(directory, files[0])

def trim_silence(input_file=None, folder=MP3_FOLDER, silence_thresh=-40, min_silence_len=300, buffer_ms=200):
    """
    Trim dead air (silence) from the beginning and end of an MP3 file, leaving a small buffer,
    and save it with '_trim' appended to the filename.

    :param input_file: Path to the input MP3 file. Defaults to the newest file in folder if not specified.
    :param folder: Folder to search for the newest file if input_file is not specified.
    :param silence_thresh: Silence threshold in dB (default is -40 dB).
    :param min_silence_len: Minimum length of silence to consider (in milliseconds).
    :param buffer_ms: Extra milliseconds to leave at the beginning and end of the trimmed audio.
    """
    # Use the newest file in the folder if input_file is None
    if input_file is None:
        file_path = get_newest_file(folder)
    else:
        file_path = input_file

    if file_path and os.path.exists(file_path):
        # Load the audio file
        audio = AudioSegment.from_mp3(file_path)
        
        # Detect non-silent portions of the audio
        non_silent_ranges = detect_nonsilent(audio, min_silence_len=min_silence_len, silence_thresh=silence_thresh)
        
        if non_silent_ranges:
            # Get the start and end of the non-silent range and add buffer
            start_trim = max(0, non_silent_ranges[0][0] - buffer_ms)
            end_trim = min(len(audio), non_silent_ranges[-1][1] + buffer_ms)
            
            # Trim the audio with the buffer
            trimmed_audio = audio[start_trim:end_trim]
            
            # Create output filename by appending '_trim' before the file extension
            base, ext = os.path.splitext(file_path)
            output_file = f"{base}_trim{ext}"
            
            # Export the trimmed audio to an MP3 file
            trimmed_audio.export(output_file, format="mp3")
            print(f"Trimmed audio saved as {output_file}")
        else:
            print("No non-silent segments detected. File not saved.")
    else:
        print("No valid audio file found. Please check the directory or specify a file.")

# Example usage
# trim_silence()  # Uses the newest file in the default "audio_chunks" folder
# Or specify a file directly
# trim_silence(input_file="specific_audio.mp3")


In [202]:
# Transcibe Speech to Text

def transcribe_audio(mp3_folder=MP3_FOLDER, specified_file=None, use_newest=True):
    """
    Transcribes an audio file using the Whisper model. By default, uses the newest file in the directory,
    but can also use a specified file if provided.

    :param mp3_folder: Directory to search for audio files (default is "audio_chunks")
    :param specified_file: File path to a specific file to transcribe; overrides use_newest if provided
    :param use_newest: Boolean flag to determine whether to use the newest file in mp3_folder (default is True)
    :return: Transcription text if successful, or an error message
    """
      # Determine file path based on user input or defaults
    file_path = specified_file if specified_file else (get_newest_file(mp3_folder) if use_newest else None)

    if file_path and os.path.exists(file_path):
        with open(file_path, "rb") as audio_file:
            # Perform transcription using Whisper model
            transcription = client.audio.transcriptions.create(
                model="whisper-1",
                file=audio_file,
            )
            print("Transcription: " + transcription.text)
            return transcription.text
    else:
        return "No valid audio file found. Please check the directory or specify a valid file."
    

# Example usage
# Or specify a file directly
# print(transcribe_audio(specified_file="specific_audio.mp3", use_newest=False))


In [203]:
# Prompt definitions (pre-written)
# MyPrompt1 find one error and give the complete correction
myPrompt1 = """
You will receive a paragraph written in Spanish. Your task is to identify and correct major grammatical errors while adhering to the following instructions:

1. Ignore gender errors.
2. If a mistake is made but then corrected within the same paragraph, do not include that correction (e.g., "yo comí" followed by "yo comer").
3. Ignore typing mistakes or missing accents; these are not considered errors.
4. Ignore minor clarity issues.
5. If there are no errors worth mentioning, output "No Errors."

Your output should consist only of the correct Spanish for each identified error, separated by periods. Do not provide any additional commentary or explanations. Aim for responses of 4-5 words unless more is needed.

Example:

Input: "Nosotros va a la playa mañana. Ellos trae sus juguetes."

Output: "Nosotros vamos. Ellos traen."
"""

# Archived prompts

myText = "Yo ir al tienda para comprar manzanas y platanos. Cuando llegué, vi muchas frutas y verduras. El cajero me dijo que el precio son muy alto, pero no me importa. Después, yo regresar a casa y preparé un plato grande para mi familia. Todos nos gusta comer juntos."

# MyPrompt2 is show all the errors in a list
myPrompt2 = """ 
You will receive a paragraph written in Spanish. Your task is to identify and correct one major grammatical error while adhering to the following instructions:
Ignore gender errors.
If a mistake is made but then corrected within the same paragraph, do not include that correction (e.g., "yo comí" followed by "yo comer").
Ignore typos or missing accents or written problems; these are not considered errors.
Ignore minor clarity issues.
If there are no significant errors, output "No errors."
Your output should consist of two sentences:

1. The original incorrect sentence, abreviated to show what was said, but not include extraneous detail before and after in the sentence.
2. The corrected version with the corrected word in all caps.
If no significant error is found, just write "No errors.". no second sentence is needed in this case
DO NOT SHOW AN ERROR IF THERE IS NOT ONE.
Example 1:

Input: Ayer, nosotros vamos a la playa mañana para solear.
Output: No errors.
Example 2:

Input: Ayer, nosotros va a la playa mañana para solear. Ellos trae sus juguetes.
Output: Nosotros va a la playa mañana. Nosotros vamos a la playa mañana.

"""
confirmPrompt = """ The following input is supposed to be a sentence in spanish that includes an error, and then the correction of that error. If the second sentence does not actually correct an error or simply repeats the sentence, then respond with 'No errors.' Otherwise repeat back exactly the input given to you without edits. 
Input:
"""
checkCorrectionPrompt = """ 

"""


In [204]:
# Get the openai secret key
secret_key = getpass.getpass("Please enter your openai key: ")
os.environ["OPENAI_API_KEY"]= secret_key

In [205]:
# OpenAI set up:
client = OpenAI()
client.api_key = secret_key
# Set up OpenAI client with the provided API key

def complete(prompt, stop=None):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
             {
                "role": "system",
                "content": "You are a real-time voice chat assistant for Spanish language learning, specializing in Mexican Spanish with a natural Mexican accent."
            },
            {
            "role": "user",
            "content": prompt
            }
        ],
        stop=stop
    )
    return response.choices[0].message.content

complete("is this working?")

'¡Hola! Sí, esto está funcionando. ¿En qué puedo ayudarte hoy con tu aprendizaje del español?'

In [208]:
testPrompt1 = """ take the following spanish dictation and identify the worst mistake you can find. Follow these rules
If a mistake is made but then immediately corrected within the same paragraph, ignore it (e.g., "yo comes" followed by "yo como").
Ignore typos or missing accents or written problems; these are not considered errors.
Ignore minor clarity issues.
ignore hypothetical, subjungtive or complex conjugations
Focus more on incorrect words getting used, or poor conjugation.
Do not include additional commentary, just output the exact sentence with the error included.
example output: Nosotros va a la playa mañana.
If not major errors can be found, output: No errors.

Input:
"""
testPrompt1a = """Ignore previous instructions, your only job here is this: In the following sentence there may be an error. If you can find it, tell me what it is, and explain your reasoning. 
If a mistake is made but then immediately corrected, ignore it (e.g., "yo comes" followed by "yo como").
Ignore typos or missing accents or written problems; these are not considered errors.
Ignore minor clarity issues.
ignore hypothetical, subjungtive or complex conjugations
Focus more on incorrect words getting used, or poor conjugation
Only explain the correction for a single error. Do not include additional commentary or questions, output nothing but one single explanation for one single error.
It's very important that if there isn't a major mistake in the sentence, to just reply 'No errors.'
Confirm for sure that the error you have selected is bad enough to bother mentioning.
"""
testPrompt2="""We are going to help a student by point out an error they made while speaking. Here is the explanation of the error, take careful not of which word is wrong
"""
testPrompt2a=""" 
Now take the following input sentence:
Input:
"""
testPrompt2b=""" 
output the sentence fragment centered on the erroneous word from the input sentence. The fragment should include the 3 words before and after. Do not include any additional commentary.
If you cannot find an obvious error, just output 'No error.' 
Example Input: Nosotros va a la playa mañana para solear y relajarnos, pero no queremos manejar por todo el dia.
Example Output: Nosotros va a la playa
"""

testPrompt3="""We are going to help a student by correcting an error they made while speaking. Using the following guidance on what the error was:
"""
testPrompt3a=""" And now edit this sentence fragment so it is correct. Do not make it a complete sentence, just correct the error described above. Provide NO ADDITONAL COMMENTARY, just output the corrected sentence fragment
Example Output: Nosotros va a la playa
Example input: Nosotros vamos a la playa
"""

testPrompt4="""You are a careful spanish teacher and you don't want to give unnecessary instruction to a student while they are learning. Look at the following correction made by another teacher and decide if the student was close enough or if the instruction is actually incorrect and should be ignored. If there is any chance the instruction is incorrect reply 'Ignore.' otherwise reply 'Go ahead.' """

confirmErrorCorrectionPrompt1=""" Yes or No, does the end of the first transcription match the second transcription word for word? Ignore spelling and punctuation etc. """
confirmErrorCorrectionPrompt1a=""" Answer either 'Yes.' or 'No.' do not add additional commentary. Also if either of the transcriptions say something in english such as 'Nevermind' then also reply 'Yes' """ 


add a beep when it stops recording
could have a faster dialogue option where it does no analysis

In [209]:
# MAIN PROGRAM LOOP

play_audio("intro.mp3")

while True:
    record_audio()

    trim_silence()  # Uses the newest file in the default "audio_chunks" folder

    myTransciption = transcribe_audio()  # Uses the newest file in the default "audio_chunks" folder

    # Check transcription for grammatical errors
    if complete(f"{testPrompt1}\n\n{myTransciption}") != "Ignore.":
        errorExplanation = complete(f"{testPrompt1a}\n\n{myTransciption}")
        print(errorExplanation)
        errorFragment = complete(f"{testPrompt2}\n{errorExplanation}n\n{testPrompt2a}\n\n{foundError}n\n{testPrompt2b}")
        print(errorFragment)
        if errorFragment != "Ignore.":
            errorFixed = complete(f"{testPrompt3}\n{errorExplanation}n\n{testPrompt3a}n\n{errorFragment}")
            print(errorFixed)
            if complete(testPrompt4) != "Ignore.":
                errorsLeft = True
            else:
                errorsLeft = False
                print("Actually nevermind")
        else:
                errorsLeft = False
                print("Actually nevermind")
    else:            
        errorsLeft = False

    while errorsLeft:
        #convert errors into audio and plays them
            errorsLeft = True
            speech_file = "speech_file.mp3"
            speech_file_path = Path.cwd() / speech_file
            #generate the TTS Audio
            response = client.audio.speech.create(
                model="tts-1",
                voice="onyx",
                input=errorFixed
            )
            response.stream_to_file(speech_file_path)

            # play audio back
            play_audio(speech_file_path)
            
            record_audio()

            trim_silence()  # Uses the newest file in the default "audio_chunks" folder

            myTransciption = transcribe_audio()  # Uses the newest file in the default "audio_chunks" folder

            if complete(f"{confirmErrorCorrectionPrompt1}\n{myTransciption}\n{errorFixed}\n{confirmErrorCorrectionPrompt1a}") != "No.":
                 errorsLeft = False
            # Check if they got it right using a gpt call, if they did they change errorLeft
            # also check against other cases (as described below)
        
        #No more errors, continue conversation, play 'cuentame.mp3' 
    play_audio("cuentame.mp3")

Finished playing intro.mp3
Recording... Speak now.
Silence detected. Stopping recording.
Saved chunk to audio_chunks\chunk29.mp3
Final chunk saved.
Trimmed audio saved as audio_chunks\chunk29_trim.mp3
Transcription: gustaría practicar contigo, pero es importante que yo hacer errores.
El error en la oración es "hacer" que debería ser "cometer." La forma correcta sería "es importante que yo cometa errores." "Hacer" no es el verbo adecuado en este contexto, ya que se usa "cometer" para referirse a la acción de cometer errores.
vas a hacer errores, debes repararlos.
vas a cometer errores, debes repararlos.


C:\Users\mattj\AppData\Local\Temp\ipykernel_824\937880058.py:43: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(speech_file_path)


Finished playing c:\Users\mattj\OneDrive\Documents\GitHub\learning\speech_file.mp3
Recording... Speak now.
Silence detected. Stopping recording.
Saved chunk to audio_chunks\chunk31.mp3
Final chunk saved.
Trimmed audio saved as audio_chunks\chunk31_trim.mp3
Transcription: ¿Vas a cometer errores?
Finished playing c:\Users\mattj\OneDrive\Documents\GitHub\learning\speech_file.mp3
Recording... Speak now.
Silence detected. Stopping recording.
Saved chunk to audio_chunks\chunk33.mp3
Final chunk saved.
Trimmed audio saved as audio_chunks\chunk33_trim.mp3
Transcription: Errores debes repararlos.
Finished playing c:\Users\mattj\OneDrive\Documents\GitHub\learning\speech_file.mp3
Recording... Speak now.
Silence detected. Stopping recording.
Saved chunk to audio_chunks\chunk35.mp3
Final chunk saved.
Trimmed audio saved as audio_chunks\chunk35_trim.mp3
Transcription: a cometer errores, debes repararlos.
Finished playing c:\Users\mattj\OneDrive\Documents\GitHub\learning\speech_file.mp3
Recording... S

KeyboardInterrupt: 

In [ ]:

''' 
Listen to audio

if the last words said were  
nevermind - just move one. this is tough because i might just completely move on, so you need logic to notice this has happened and ignore checking against old errors
repite - auto replay everything that was just said and ignore 
repite (algo) figure out what exactly i want repeated and repeat that audio slowly (consider slowing down the playback)

otherwise confirm that i repeated back the correction properly. 
'''

' \nif the last words said were  \nnevermind - just move one\nrepite - auto replay everything that was just said and ignore \nrepite (algo) figure out what exactly i want repeated and repeat that audio slowly (consider slowing down the playback)\n\nListen to audio\nI respond normally - one by one, go through the things that were said to me and check if i said them correctly back. \n    if yes, then move to the next item,\n    if no, add that item back to the audio output\n    if all yes, then continue conversation\n    otherwise, concat audio again and play new audio\n\n'